# Importing libraries

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import os
import requests

## Importing Data

In [2]:
path = r'C:\Users\smerg\esport_earning_2018-2023'

data = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'EsportsData.cvs'))

In [3]:
# Alternative URL for World Countries GeoJSON hosted on GitHub
geojson_url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'

# Load the GeoJSON file
response = requests.get(geojson_url)

# Check if the response status code is 200 (OK)
if response.status_code == 200:
    geo_data = response.json()
    # Inspect the structure of the GeoJSON data
    print(geo_data['features'][0]['properties'])
else:
    print(f"Failed to load GeoJSON. Status code: {response.status_code}")


{'name': 'Afghanistan'}


In [4]:
data.head()

Unnamed: 0            Game ReleaseDate     Genre  TotalEarnings  \
0           0  Age of Empires  1997-01-01  Strategy      510716.46   
1           1  Age of Empires  1997-01-01  Strategy      510716.46   
2           2  Age of Empires  1997-01-01  Strategy      510716.46   
3           3  Age of Empires  1997-01-01  Strategy      510716.46   
4           4  Age of Empires  1997-01-01  Strategy      510716.46   

   OfflineEarnings  PercentOffline  TotalPlayers  TotalTournaments Country  \
0        375857.63        0.735942           473               265     NaN   
1        375857.63        0.735942           473               265     NaN   
2        375857.63        0.735942           473               265     NaN   
3        375857.63        0.735942           473               265     NaN   
4        375857.63        0.735942           473               265     NaN   

         Date  Earnings  Players  Tournaments  
0  1998-12-01   2000.00      1.0          1.0  
1  2000-04-01   9500.00      4.0          1.0  
2  2002-09-01    100.00      3.0          1.0  
3  2003-05-01    625.00      4.0          1.0  
4  2006-08-01     42.25     10.0          1.0

In [5]:
# Check for missing values in the dataset
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)

# Handle missing values
# Example: Fill missing values in 'TotalEarnings' with 0 (if this makes sense for your analysis)
data['TotalEarnings'].fillna(0, inplace=True)

# Drop rows with missing location data (e.g., missing 'Country' entries)
data.dropna(subset=['Country'], inplace=True)


Missing values:
 Unnamed: 0             0
Game                   0
ReleaseDate            0
Genre                  0
TotalEarnings          0
OfflineEarnings        0
PercentOffline         0
TotalPlayers           0
TotalTournaments       0
Country             8740
Date                  53
Earnings              53
Players               53
Tournaments           53
dtype: int64


C:\Users\smerg\AppData\Local\Temp\ipykernel_9160\384934009.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TotalEarnings'].fillna(0, inplace=True)


In [6]:
# Ensure numerical columns are in the correct format
data['TotalEarnings'] = pd.to_numeric(data['TotalEarnings'], errors='coerce')
data['TotalPlayers'] = pd.to_numeric(data['TotalPlayers'], errors='coerce')

# Ensure dates (if any) are in the correct datetime format
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Confirm data types
print(data.dtypes)


Unnamed: 0                   int64
Game                        object
ReleaseDate                 object
Genre                       object
TotalEarnings              float64
OfflineEarnings            float64
PercentOffline             float64
TotalPlayers                 int64
TotalTournaments             int64
Country                     object
Date                datetime64[ns]
Earnings                   float64
Players                    float64
Tournaments                float64
dtype: object


In [7]:
# Print unique country names in your dataset
print(data['Country'].unique())

# Example of renaming countries to match the GeoJSON data (if needed)
data['Country'] = data['Country'].replace({
    'USA': 'United States of America',
    'South Korea': 'Korea, Republic of',
    # Add more replacements as necessary
})


['USA' 'South Korea' 'China']


In [8]:
# Filter for relevant data (e.g., earnings after 2018 or only specific regions)
# Example: Filter for years after 2018
if 'Year' in data.columns:
    data = data[data['Year'] >= 2018]

# Ensure the filtered dataset is clean
print(data.head())


      Unnamed: 0          Game ReleaseDate                 Genre  \
1987        1987  Call of Duty  2003-01-01  First-Person Shooter   
1988        1988  Call of Duty  2003-01-01  First-Person Shooter   
1989        1989  Call of Duty  2003-01-01  First-Person Shooter   
3044        3044     Overwatch  2016-01-01  First-Person Shooter   
3045        3045     Overwatch  2016-01-01  First-Person Shooter   

      TotalEarnings  OfflineEarnings  PercentOffline  TotalPlayers  \
1987       54799.77         54799.77        1.000000            66   
1988       54799.77         54799.77        1.000000            66   
1989       54799.77         54799.77        1.000000            66   
3044    34616725.38      18840032.08        0.544247          4047   
3045    34616725.38      18840032.08        0.544247          4047   

      TotalTournaments                   Country       Date  Earnings  \
1987                 4  United States of America 2004-08-01  50000.00   
1988                 4  

In [9]:
# Inspect the cleaned data
print(data.head())

# Save the cleaned data if needed
cleaned_data_path = '/mnt/data/EsportsDataCleanedReady.csv'
data.to_csv(cleaned_data_path, index=False)


      Unnamed: 0          Game ReleaseDate                 Genre  \
1987        1987  Call of Duty  2003-01-01  First-Person Shooter   
1988        1988  Call of Duty  2003-01-01  First-Person Shooter   
1989        1989  Call of Duty  2003-01-01  First-Person Shooter   
3044        3044     Overwatch  2016-01-01  First-Person Shooter   
3045        3045     Overwatch  2016-01-01  First-Person Shooter   

      TotalEarnings  OfflineEarnings  PercentOffline  TotalPlayers  \
1987       54799.77         54799.77        1.000000            66   
1988       54799.77         54799.77        1.000000            66   
1989       54799.77         54799.77        1.000000            66   
3044    34616725.38      18840032.08        0.544247          4047   
3045    34616725.38      18840032.08        0.544247          4047   

      TotalTournaments                   Country       Date  Earnings  \
1987                 4  United States of America 2004-08-01  50000.00   
1988                 4  

OSError: Cannot save file into a non-existent directory: '\mnt\data'

In [10]:
# Inspect the cleaned data
print(data.head())

# Save the cleaned data if needed
cleaned_data_path = '/mnt/data/EsportsDataCleanedReady.csv'
data.to_csv(cleaned_data_path, index=False)


      Unnamed: 0          Game ReleaseDate                 Genre  \
1987        1987  Call of Duty  2003-01-01  First-Person Shooter   
1988        1988  Call of Duty  2003-01-01  First-Person Shooter   
1989        1989  Call of Duty  2003-01-01  First-Person Shooter   
3044        3044     Overwatch  2016-01-01  First-Person Shooter   
3045        3045     Overwatch  2016-01-01  First-Person Shooter   

      TotalEarnings  OfflineEarnings  PercentOffline  TotalPlayers  \
1987       54799.77         54799.77        1.000000            66   
1988       54799.77         54799.77        1.000000            66   
1989       54799.77         54799.77        1.000000            66   
3044    34616725.38      18840032.08        0.544247          4047   
3045    34616725.38      18840032.08        0.544247          4047   

      TotalTournaments                   Country       Date  Earnings  \
1987                 4  United States of America 2004-08-01  50000.00   
1988                 4  

OSError: Cannot save file into a non-existent directory: '\mnt\data'

In [11]:
# Check for missing values and extreme values in the dataset
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)

# Check for extreme values in numeric columns like 'TotalEarnings' and 'TotalPlayers'
# We'll use the describe function to identify any unusually high or low values
data_describe = data.describe()

data_describe


Missing values:
 Unnamed: 0          0
Game                0
ReleaseDate         0
Genre               0
TotalEarnings       0
OfflineEarnings     0
PercentOffline      0
TotalPlayers        0
TotalTournaments    0
Country             0
Date                0
Earnings            0
Players             0
Tournaments         0
dtype: int64


Unnamed: 0  TotalEarnings  OfflineEarnings  PercentOffline  \
count   557.000000   5.570000e+02     5.570000e+02      557.000000   
mean   5505.425494   1.484256e+08     1.151840e+08        0.708178   
min    1987.000000   5.479977e+04     5.479977e+04        0.286745   
25%    4121.000000   3.461673e+07     1.884003e+07        0.544247   
50%    4424.000000   1.050111e+08     9.081567e+07        0.864820   
75%    8545.000000   3.371080e+08     2.982906e+08        0.884852   
max    8932.000000   3.371080e+08     2.982906e+08        1.000000   
std    2215.574849   1.205331e+08     1.123911e+08        0.206102   

       TotalPlayers  TotalTournaments                           Date  \
count    557.000000        557.000000                            557   
mean    5934.960503       1835.267504  2018-03-12 17:45:08.078994688   
min       66.000000          4.000000            2004-08-01 00:00:00   
25%     4047.000000       1124.000000            2015-11-01 00:00:00   
50%     4784.000000       1841.000000            2018-08-01 00:00:00   
75%     9211.000000       2895.000000            2020-12-01 00:00:00   
max     9211.000000       2895.000000            2023-10-01 00:00:00   
std     2462.434876        755.817642                            NaN   

           Earnings      Players  Tournaments  
count  5.570000e+02   557.000000   557.000000  
mean   1.214377e+06   227.405745    15.149013  
min    2.500000e+01     0.000000     1.000000  
25%    1.051603e+05    82.000000     7.000000  
50%    3.478492e+05   188.000000    12.000000  
75%    1.065899e+06   317.000000    21.000000  
max    4.013865e+07  1402.000000    97.000000  
std    3.626997e+06   198.234792    11.716574

In [12]:
# Check for missing values in the dataset
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)

# Option 1: Drop rows with missing values (if the missing data is important)
data_cleaned = data.dropna()

# Option 2: Fill missing values with a default value (e.g., 0 for numerical columns)
data_cleaned = data.fillna(0)

# Confirm that missing values have been handled
print("Missing values after cleaning:\n", data_cleaned.isnull().sum())


Missing values:
 Unnamed: 0          0
Game                0
ReleaseDate         0
Genre               0
TotalEarnings       0
OfflineEarnings     0
PercentOffline      0
TotalPlayers        0
TotalTournaments    0
Country             0
Date                0
Earnings            0
Players             0
Tournaments         0
dtype: int64
Missing values after cleaning:
 Unnamed: 0          0
Game                0
ReleaseDate         0
Genre               0
TotalEarnings       0
OfflineEarnings     0
PercentOffline      0
TotalPlayers        0
TotalTournaments    0
Country             0
Date                0
Earnings            0
Players             0
Tournaments         0
dtype: int64


In [13]:
# Use the describe function to get an overview of numerical data
data_description = data.describe()
print(data_description)

# Optionally, you can identify and remove extreme outliers using IQR or Z-score methods
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Filter out the outliers
data_cleaned = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]

# Check the dataset after outlier removal
print(data_cleaned.describe())


        Unnamed: 0  TotalEarnings  OfflineEarnings  PercentOffline  \
count   557.000000   5.570000e+02     5.570000e+02      557.000000   
mean   5505.425494   1.484256e+08     1.151840e+08        0.708178   
min    1987.000000   5.479977e+04     5.479977e+04        0.286745   
25%    4121.000000   3.461673e+07     1.884003e+07        0.544247   
50%    4424.000000   1.050111e+08     9.081567e+07        0.864820   
75%    8545.000000   3.371080e+08     2.982906e+08        0.884852   
max    8932.000000   3.371080e+08     2.982906e+08        1.000000   
std    2215.574849   1.205331e+08     1.123911e+08        0.206102   

       TotalPlayers  TotalTournaments                           Date  \
count    557.000000        557.000000                            557   
mean    5934.960503       1835.267504  2018-03-12 17:45:08.078994688   
min       66.000000          4.000000            2004-08-01 00:00:00   
25%     4047.000000       1124.000000            2015-11-01 00:00:00   
50%     4

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [14]:
# Save the cleaned data
data_cleaned.to_csv('CleanedEsportsData.csv', index=False)


In [15]:
# Load the GeoJSON file with country boundaries
geojson_url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'

# Request and load the GeoJSON file
geo_data = requests.get(geojson_url).json()

# Inspect the GeoJSON structure
print(geo_data['features'][0]['properties'])  # This shows the available properties for each country

{'name': 'Afghanistan'}


In [17]:
# Inspect the GeoJSON structure
print(geo_data['features'][0]['properties'])


{'name': 'Afghanistan'}


In [18]:
# Create a base map centered on the world
m = folium.Map(location=[20, 0], zoom_start=2)

# Add a choropleth layer
folium.Choropleth(
    geo_data=geo_data,
    name="choropleth",
    data=data,
    columns=['Country', 'TotalEarnings'],
    key_on="feature.properties.name",  # Update to the correct key
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Earnings by Country"
).add_to(m)

# Add a layer control for the map
folium.LayerControl().add_to(m)

# Display the map
m


In [19]:
# Example: Adjust country names in your dataset to match the GeoJSON file
data['Country'] = data['Country'].replace({
    'United States': 'United States of America',
    'South Korea': 'Korea, Republic of',
    # Add more replacements if needed
})


In [21]:
# Modify the 'key_on' argument to use the correct property key
folium.Choropleth(
    geo_data=geo_data,
    name="choropleth",
    data=data,
    columns=['Country', 'TotalEarnings'],
    key_on="feature.properties.name",  # Use the correct key here
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Earnings by Country"
).add_to(m)


In [22]:
# Check the country names in your dataset and the GeoJSON
print(data['Country'].unique())  # From your dataset
print([feature['properties']['name'] for feature in geo_data['features']][:10])  # From GeoJSON (based on the correct key)


['United States of America' 'Korea, Republic of' 'China']
['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates', 'Argentina', 'Armenia', 'Antarctica', 'French Southern and Antarctic Lands', 'Australia', 'Austria']


In [23]:
# Standardize country names to match the GeoJSON
data['Country'] = data['Country'].replace({
    'USA': 'United States of America',
    'South Korea': 'Korea, Republic of'
})


In [25]:
# Save the map to an HTML file
m.save('choropleth_map.html')


## Discussion of Results
The choropleth map we created provided a visual representation of Total Earnings by Country in the esports industry. The countries are shaded based on their total earnings, with darker colors indicating higher earnings.

### Key Observations:
Dominant Countries:

Countries such as United States, China, and South Korea are highlighted as major earners in the esports industry, which aligns with their established gaming infrastructures and large player bases.
These countries are known for hosting some of the largest esports tournaments, and they have a strong history of competitive gaming, which contributes to their dominance in total earnings.
Emerging Markets:

Countries like Brazil and Germany also show significant earnings, although they are not as prominent as the top-tier countries.
These regions represent emerging markets in esports, and the data suggests that there is potential for further growth as the industry continues to globalize.
Low-Earning Regions:

Countries in Africa, some parts of Latin America, and Southeast Asia show lower earnings. This may be due to a lack of infrastructure, limited investment in esports, or smaller competitive gaming communities.
#### Answering Research Questions:
Which countries are home to the most profitable esports games?

The analysis confirms that United States, China, and South Korea are the most profitable countries in the esports industry. This matches expectations and highlights the importance of these regions in driving the growth of competitive gaming globally.
Are there specific geographic regions that are emerging as esports powerhouses?

Brazil and Germany are examples of emerging markets in esports. The analysis suggests that while they are not yet at the level of the leading countries, they are growing and could become more significant players in the future.
#### New Research Questions:
What factors contribute to the dominance of certain countries in esports?

Why are countries like the United States and South Korea so dominant in esports earnings? Further research could explore factors such as investment in infrastructure, player population, and sponsorships to better understand what drives their success.
What are the barriers to growth in lower-earning regions?

Many regions, particularly in Africa and parts of Southeast Asia, have low esports earnings. Investigating what barriers exist, such as lack of infrastructure, internet access, or gaming culture, could help uncover ways to promote esports development in these regions.
How do different game genres perform in different regions?

The map shows total earnings by country, but a deeper analysis could explore whether specific game genres are more popular or profitable in certain regions. This could inform game developers and tournament organizers about regional preferences.
##### Conclusion:
The choropleth map provides valuable insights into the global esports landscape, highlighting both dominant countries and emerging markets. It also raises new questions about the underlying factors that drive esports success and the potential for future growth in underrepresented regions. This analysis serves as a foundation for further research and strategic decision-making in the esports industry.